In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from my_functions import integer_perturbation, get_X_pca_totals, contains_at_least_one, add_new_elements_pca

In [2]:
df = pd.read_excel('data/IncidentiModificato.xlsx', index_col='VERBALE')
df = df.drop(columns=["DATA"])
X = df[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'BMI', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y = df['Mezzo']

df.head()

,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,Testa:Cervelletto,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,0,81,84.0,1.75,27.428571,0,1,0,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,1,69,69.0,1.62,26.291724,1,4,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,1,71,67.0,1.55,27.887617,1,2,0,1,1,...,0,0,0,0,4,6,0,0,4,10
86220,1,54,60.0,1.59,23.733238,1,4,0,0,1,...,0,0,0,0,4,5,3,2,4,14
86247,1,78,69.0,1.67,24.740937,1,2,0,0,0,...,0,0,0,0,4,2,0,2,4,8


## Divido i dati tra continui e lesioni, poi applico t-SNE sui due insiemi con n_components=3 e li unisco in un solo dataframe 

In [3]:
from sklearn.manifold import TSNE 

X_pca = get_X_pca_totals(df)

X_dati_continui = X_pca[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'BMI']]
X_dati_PCA = X_pca[['PCA Testa', 'PCA Torace', 'PCA Addome', 'PCA Scheletro']]

X_dati_continui_TSNE = TSNE(n_components=3, perplexity=50).fit_transform(X_dati_continui)
X_dati_PCA_TSNE = TSNE(n_components=3, perplexity=50).fit_transform(X_dati_PCA)

In [4]:
list_TSNE = []
for i in range(len(X_dati_continui_TSNE)):
    list_TSNE.append(np.concatenate((X_dati_continui_TSNE[i], X_dati_PCA_TSNE[i])))
df_TSNE = pd.DataFrame(np.array(list_TSNE))

In [5]:
df_TSNE.head()

,0,1,2,3,4,5
0,-22.540997,89.917305,118.186653,111.449608,117.004387,15.799403
1,30.979012,19.630531,26.148876,52.560600,157.340317,60.256168
2,-19.516668,-81.394531,16.127821,-33.638565,10.161092,-28.960627
3,59.317516,-113.269226,-66.136154,-44.602600,101.649162,6.787167
4,23.119600,-10.844270,75.101768,-48.437828,-51.640427,-23.045012


In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

params = {'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10],
          'learning_rate_init' : [0.01, 0.2, 0.001],
          'activation': ['identity', 'logistic', 'tanh', 'relu']}

mlp = MLPClassifier(max_iter=5000)

clf = GridSearchCV(mlp, params, n_jobs=-1, cv=5)
scores = cross_val_score(clf, df_TSNE, y, cv=3)
print("\nAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores


Accuracy: 0.54 (+/- 0.16)


array([0.5       , 0.46511628, 0.65116279])